In [65]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

train_dir = '/kaggle/input/herbarium-2022-fgvc9/train_images/'
test_dir = '/kaggle/input/herbarium-2022-fgvc9/test_images'

with open("/kaggle/input/herbarium-2022-fgvc9/train_metadata.json") as json_file:
    train_meta = json.load(json_file)
with open("/kaggle/input/herbarium-2022-fgvc9/test_metadata.json") as json_file:
    test_meta = json.load(json_file)

In [66]:
image_ids = [image["image_id"] for image in train_meta["images"]]
image_dirs = [train_dir + image['file_name'] for image in train_meta["images"]]
category_ids = [annotation['category_id'] for annotation in train_meta['annotations']]
genus_ids = [annotation['genus_id'] for annotation in train_meta['annotations']]

test_ids = [image['image_id'] for image in test_meta]
test_dirs = [test_dir + image['file_name'] for image in test_meta]

#Create the initial training dataframe with the above defined columns
train_df = pd.DataFrame({
    "image_id" : image_ids,
    "image_dir" : image_dirs,
    "category" : category_ids,
    "genus" : genus_ids})

#Create a testing dataframe
test_df = pd.DataFrame({
    "test_id" : test_ids,
    "test_dir" : test_dirs
})

train_df

,image_id,image_dir,category,genus
0,00000__001,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,1
1,00000__002,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,1
2,00000__003,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,1
3,00000__004,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,1
4,00000__005,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,1
...,...,...,...,...
839767,15504__032,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,2584
839768,15504__033,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,2584
839769,15504__035,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,2584
839770,15504__036,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,2584


In [67]:
#Add a genus column to the dataframe
genus_map = {genus['genus_id'] : genus['genus'] for genus in train_meta['genera']}
train_df['genus'] = train_df['genus'].map(genus_map)

##Create a family column in the dataframe based on the genus names
    # Step 1: Create dictionary of genus -> family mapping
genus_family_map = {}
for category in train_meta["categories"]:
    genus = category['genus']
    family = category['family']
    genus_family_map[genus] = family

    # Step 2: Create new column with default value of None™
train_df['family'] = None

    # Step 3: Update values in new column based on genus -> family mapping
for i, row in train_df.iterrows():
    genus = row['genus']
    if genus in genus_family_map:
        family = genus_family_map[genus]
        train_df.at[i, 'family'] = family

train_df

,image_id,image_dir,category,genus,family
0,00000__001,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,Abies,Pinaceae
1,00000__002,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,Abies,Pinaceae
2,00000__003,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,Abies,Pinaceae
3,00000__004,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,Abies,Pinaceae
4,00000__005,/kaggle/input/herbarium-2022-fgvc9/train_image...,0,Abies,Pinaceae
...,...,...,...,...,...
839767,15504__032,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,Zygophyllum,Zygophyllaceae
839768,15504__033,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,Zygophyllum,Zygophyllaceae
839769,15504__035,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,Zygophyllum,Zygophyllaceae
839770,15504__036,/kaggle/input/herbarium-2022-fgvc9/train_image...,15504,Zygophyllum,Zygophyllaceae


In [68]:
#Filter only the images of plants that are in the Poaceae family
train_df = train_df.loc[train_df['family'] == 'Poaceae']
#Reset index
train_df = train_df.reset_index(drop=True)

train_df

,image_id,image_dir,category,genus,family
0,00333__001,/kaggle/input/herbarium-2022-fgvc9/train_image...,333,Agrostis,Poaceae
1,00333__002,/kaggle/input/herbarium-2022-fgvc9/train_image...,333,Agrostis,Poaceae
2,00333__003,/kaggle/input/herbarium-2022-fgvc9/train_image...,333,Agrostis,Poaceae
3,00333__004,/kaggle/input/herbarium-2022-fgvc9/train_image...,333,Agrostis,Poaceae
4,00333__005,/kaggle/input/herbarium-2022-fgvc9/train_image...,333,Agrostis,Poaceae
...,...,...,...,...,...
53542,15501__101,/kaggle/input/herbarium-2022-fgvc9/train_image...,15501,Zuloagaea,Poaceae
53543,15501__103,/kaggle/input/herbarium-2022-fgvc9/train_image...,15501,Zuloagaea,Poaceae
53544,15501__105,/kaggle/input/herbarium-2022-fgvc9/train_image...,15501,Zuloagaea,Poaceae
53545,15501__106,/kaggle/input/herbarium-2022-fgvc9/train_image...,15501,Zuloagaea,Poaceae


In [69]:
train_df["genus"].value_counts()

Muhlenbergia       4228
Paspalum           3124
Poa                2608
Dichanthelium      2474
Sporobolus         2304
                   ... 
Ptilagrostiella      14
Rhipidocladum        11
Dupontia             10
Kalinia              10
Barkworthia           8
Name: genus, Length: 158, dtype: int64

# Model creating

In [72]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from PIL import Image

!pip install fvcore

In [73]:
df = train_df
# Split the dataset for each class separately
train_dfs = []
val_dfs = []
for label in df['genus'].unique():
    # Filter the dataset to only include images with the current label
    label_df = df[df['genus'] == label]
    
    # Split the dataset into training and evaluation sets
    train_df, val_df = train_test_split(label_df, test_size=0.2)
    
    # Append the training and evaluation sets to their respective lists
    train_dfs.append(train_df)
    val_dfs.append(val_df)

# Concatenate the training and evaluation sets for all classes into single DataFrames
train_df = pd.concat(train_dfs)
val_df = pd.concat(val_dfs)

train_df['genus'] = pd.factorize(train_df['genus'])[0]
val_df['genus'] = pd.factorize(val_df['genus'])[0]

In [74]:
batch_size = 128
epochs = 4
IM_SIZE = 224
learning_rate = 1e-3

X_train, Y_train = train_df["image_dir"].values, train_df["genus"].values

X_val, Y_val = val_df["image_dir"].values, val_df["genus"].values

Transform = transforms.Compose([
     transforms.Resize((IM_SIZE, IM_SIZE)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [75]:
class GetData(Dataset):
    def __init__(self, FNames, Labels, Transform):
        self.fnames = FNames
        self.transform = Transform
        self.labels = Labels         
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):       
        x = Image.open(self.fnames[index])
    
        if "train" in self.fnames[index]:             
            return self.transform(x), self.labels[index]
        elif "test" in self.fnames[index]:            
            return self.transform(x), self.fnames[index]

                
trainset = GetData(X_train, Y_train, Transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

valset = GetData(X_val, Y_val, Transform)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=True)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = torch.hub.load('zhanghang1989/ResNeSt', 'resnest101', pretrained=True)            

Using cache found in /root/.cache/torch/hub/zhanghang1989_ResNeSt_master


In [76]:
num_classes = train_df['genus'].nunique()
class_counts = np.bincount(Y_train)

print(num_classes)
print(class_counts)

158
[ 706   11  240   12  121  739  117   60  118   16 1560   48  112   57
  216    6   44   83  405 1141  124 1166  629   60  461   24   56  200
  407   14  116   22  252   56   59  104  422  192   90 1979  696   52
   53  115    8  448   32  135  978   57 1654  397   60  995  120  240
 1175  684  124  209   57   61   53   33  199  266   53  219   62  141
   71   40   52    8   58  116  283  288  471  371   56   60  117  752
   36   56   48 3382  124  367   14   57  172   74   43   62   26 1132
  164   60 2499   59  262  149   44   54   61  219  103 2086   76  132
   60   56   11   11    8  168  228   60   54  298   57   56   39 1029
   16   32  232   56  328 1843  130  149   57   29   11   21   57   55
   24  329   52   55  120   36  111   45   15   70  534   37   30   16
   46   80   48   48]


In [77]:
total_layers = len(list(model.parameters()))

for param in model.parameters():
    param.requires_grad = True
    
n_inputs = model.fc.in_features
last_layer = nn.Linear(n_inputs, num_classes)
model.fc = last_layer 
if torch.cuda.is_available():
    model.cuda()
print(model.fc.out_features)    

if torch.cuda.is_available():
    class_weights = class_weights.cuda()

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)

158


In [78]:
"""X = train_df['image_dir'].values
Y = train_df['genus'].values

# Split the dataset into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

# Create the training and validation datasets
train_dataset = CustomDataset(X_train, Y_train, transform=data_transforms, transform_no_aug=transform_no_aug, min_samples=20)
val_dataset = CustomDataset(X_val, Y_val, transform=transform_no_aug)  # Apply only the non-augmented transform to validation set

# Create the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

print("Size of the training dataset:", len(X_val))"""

'X = train_df[\'image_dir\'].values\nY = train_df[\'genus\'].values\n\n# Split the dataset into training and validation sets\nX_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)\n\n# Create the training and validation datasets\ntrain_dataset = CustomDataset(X_train, Y_train, transform=data_transforms, transform_no_aug=transform_no_aug, min_samples=20)\nval_dataset = CustomDataset(X_val, Y_val, transform=transform_no_aug)  # Apply only the non-augmented transform to validation set\n\n# Create the DataLoaders\ntrain_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)\nval_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)\n\nprint("Size of the training dataset:", len(X_val))'

In [79]:
from tqdm import tqdm
from sklearn.metrics import f1_score

def train(trainloader, model, criterion, optimizer, scaler, device=torch.device("cpu")):
    train_acc = 0.0
    train_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(trainloader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast(enabled=True):
            output = model(images)
            loss = criterion(output, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            acc = ((output.argmax(dim=1) == labels).float().mean())
            train_acc += acc
            train_loss += loss
            y_true += labels.cpu().numpy().tolist()
            y_pred += output.argmax(dim=1).cpu().numpy().tolist()
            
    train_f1 = f1_score(y_true, y_pred, average=None)
    train_f1_avg = f1_score(y_true, y_pred, average='macro')
    return train_acc/len(trainloader), train_loss/len(trainloader), train_f1, train_f1_avg

In [80]:
def evaluate(testloader, model, criterion, device=torch.device("cpu")):
    eval_acc = 0.0
    eval_loss = 0.0
    y_true = []
    y_pred = []
    for images, labels in tqdm(testloader):
        images = images.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            output = model(images)
            loss = criterion(output, labels)
        acc = ((output.argmax(dim=1) == labels).float().mean())
        eval_acc += acc
        eval_loss += loss
        y_true += labels.cpu().numpy().tolist()
        y_pred += output.argmax(dim=1).cpu().numpy().tolist()
  
    eval_f1 = f1_score(y_true, y_pred, average=None)
    eval_f1_avg = f1_score(y_true, y_pred, average='macro')
    return eval_acc/len(testloader), eval_loss/len(testloader), eval_f1, eval_f1_avg

In [81]:
scaler = torch.cuda.amp.GradScaler(enabled=True)

train_f1_scores = []  # Initialize an empty list to store training F1 scores
val_f1_scores = []  # Initialize an empty list to store validation F1 scores
train_losses = []
val_losses = []


for epoch in range(epochs):
    train_acc, train_loss, train_f1, train_f1_avg = train(trainloader, model, criterion, optimizer, scaler, device=device)
    eval_acc, eval_loss, eval_f1, eval_f1_avg = evaluate(valloader, model, criterion, device=torch.device("cuda"))

    train_f1_scores.append(train_f1_avg)  # Store the training F1 score for this epoch
    val_f1_scores.append(eval_f1_avg)  # Store the validation F1 score for this epoch
    train_losses.append(train_loss)
    val_losses.append(eval_loss)

    print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss} | Train F1 (Avg): {train_f1_avg}")
    print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss} | Val F1 (Avg): {eval_f1_avg}")
    
    print("F1 score per class (Train):")
    for i, f1 in enumerate(train_f1):
        print(f"Class {i}: {f1}")

    print("\nF1 score per class (Validation):") 
    for i, f1 in enumerate(eval_f1):
        print(f"Class {i}: {f1}")

    print("===="*8)

  0%|          | 0/335 [00:04<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB (GPU 0; 15.90 GiB total capacity; 14.85 GiB already allocated; 53.75 MiB free; 14.97 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# Assuming `train_f1_scores` and `val_f1_scores` are the lists with the F1 scores for each epoch
epochs_range = range(1, 3 + 1)

plt.figure(figsize=(10, 5))
plt.plot(epochs_range, train_f1_scores, label='Training F1')
plt.plot(epochs_range, val_f1_scores, label='Validation F1')
plt.xlabel('Epochs')
plt.ylabel('F1 Score')
plt.title('F1 Score vs. Epochs')
plt.legend()
plt.show()

plt.plot(epochs_range, train_losses, label="Training Loss")
plt.plot(epochs_range, val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss Over Epochs")
plt.legend()
plt.show()

#plt.plot(np.arange(len(training_losses))-0.5, training_losses)
#plt.plot(validation_losses)



In [ ]:
!pip install grad-cam
from grad_cam import GradCAM

# Create a GradCAM object for your model
gradcam = GradCAM(model=model, feature_module=model.layer4, target_layer_names=["2"], use_cuda=True)

# Choose an image from the validation set
image, label = valset[0]
image = image.unsqueeze(0).to(device)  # Add a batch dimension

# Get the class activation map
cam = gradcam(image)

# Visualize the results
import matplotlib.pyplot as plt
import cv2

def show_cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

image = image.squeeze(0).cpu().numpy().transpose(1, 2, 0)  # Convert the image back to the original shape
image = image * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])  # Denormalize the image
image = np.clip(image, 0, 1)

cam_image = show_cam_on_image(image, cam)

plt.imshow(cam_image)
plt.show()

In [ ]:
"""scaler2 = torch.cuda.amp.GradScaler(enabled=True)
epochs2 = 10
for epoch in range(epochs2):
    train_acc, train_loss, train_f1, train_f1_avg = train(trainloader, model, criterion, optimizer, scaler, device=device)
    eval_acc, eval_loss, eval_f1, eval_f1_avg = evaluate(valloader, model, criterion, device=torch.device("cuda"))

    print(f"Epoch {epoch + 1} | Train Acc: {train_acc*100} | Train Loss: {train_loss} | Train F1 (Avg): {train_f1_avg}")
    print(f"\t Val Acc: {eval_acc*100} | Val Loss: {eval_loss} | Val F1 (Avg): {eval_f1_avg}")
    
    print("F1 score per class (Train):")
    for i, f1 in enumerate(train_f1):
        print(f"Class {i}: {f1}")

    print("\nF1 score per class (Validation):") 
    for i, f1 in enumerate(eval_f1):
        print(f"Class {i}: {f1}")

    print("===="*8)"""